In [1]:
from scipy import ndimage, sparse
from scipy.linalg import eigh, inv, logm, norm
import scipy.sparse
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob

In [2]:
def edges_rescaling(edges,scale): # edges are mat.data where mat is a sparse scipy matrix
    edges = np.log10(edges) # log rescale weights because they vary over many decades
    edges -= min(edges) # make them positive 
    edges /= max(edges)*1.0/scale # rescale from 0 to scale
    return edges

def build_omegaij(K,m):
    #set the initial omegaIJ
    nzr = K.row # list of nonzero rows and cols pairs
    nzc = K.col # list of nonzero rows and cols pairs
    omegaIJ_data = np.zeros(K.data.shape)
    omegaIJ_data = np.asfarray([K.data[ind]*(1.0/m[nzr[ind]] + 1.0/m[nzc[ind]]) for ind in range(omegaIJ_data.shape[0])])
    omegaIJ = sparse.coo_matrix((omegaIJ_data, (nzr, nzc)), shape=K.shape)
    return omegaIJ
def build_omegai(Kr,m):
    #set the initial omegaI
    omegaI = np.divide(Kr,m.reshape((m.shape[0],1))) #since m is ones leave it implicit
    return omegaI
def remove_col(mat,index_to_drop): #csr
    to_keep = list(set(range(mat.shape[1]))-set(index_to_drop))    
    mat = mat[:,to_keep]
    return mat
def remove_row(mat,index_to_drop): #csc
    to_keep = list(set(range(mat.shape[0]))-set(index_to_drop))    
    mat = mat[to_keep,:]
    return mat
def remove_2nodes(mat,nodes):
    mat = mat.tocoo()
    todrop1 = np.logical_or((mat.row==nodes[0]),(mat.row==nodes[1])).nonzero()[0]
    todrop2 = np.logical_or((mat.col==nodes[0]),(mat.col==nodes[1])).nonzero()[0]
    todrop = list(set(np.concatenate((todrop1,todrop2))))
    newdata=np.delete(mat.data,todrop)
    newrow=np.delete(mat.row,todrop)
    newcol=np.delete(mat.col,todrop)
    return sparse.coo_matrix((newdata, (newrow, newcol)), shape=mat.shape)
def remove_1node(mat,node):
    mat = mat.tocoo()
    todrop = np.logical_or((mat.row==node[0]),(mat.col==node[0])).nonzero()[0]
    todrop = list(set(todrop))
    newdata=np.delete(mat.data,todrop)
    newrow=np.delete(mat.row,todrop)
    newcol=np.delete(mat.col,todrop)
    return sparse.coo_matrix((newdata, (newrow, newcol)), shape=mat.shape)
def expand(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,idxs,m,g):
    for idx in idxs:
        newdata=K.data[idx]
        j=K.col[idx]
        #add (g,j,newdata)
        Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol = expand1(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,newdata,m,g,j)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol
def expand1(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,newdata,m,g,j):
    Kdata = np.append(Kdata,newdata)
    Krow = np.append(Krow,g)
    Kcol = np.append(Kcol,j)
    omegaIJdata = np.append(omegaIJdata,newdata*(1.0/m[-1]+1.0/m[j]))
    omegaIJcol = np.append(omegaIJcol,g)
    omegaIJrow = np.append(omegaIJrow,j)
    #add symmetric
    Kdata = np.append(Kdata,newdata)
    Krow = np.append(Krow,j)
    Kcol = np.append(Kcol,g)
    omegaIJdata = np.append(omegaIJdata,newdata*(1.0/m[-1]+1.0/m[j]))
    omegaIJcol = np.append(omegaIJcol,j)
    omegaIJrow = np.append(omegaIJrow,g)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol
def delete_nodes(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,idxs):
    Kdata = np.delete(Kdata,idxs)
    Krow = np.delete(Krow,idxs)
    Kcol = np.delete(Kcol,idxs)
    omegaIJdata = np.delete(omegaIJdata,idxs)
    omegaIJrow = np.delete(omegaIJrow,idxs)
    omegaIJcol = np.delete(omegaIJcol,idxs)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol

In [3]:
K0 = sparse.load_npz('/home/garner1/github/tissue2graph/npz/mat_XY_10nn.npz')

In [4]:
N = 10
K = K0.copy()[:N,:N].tocoo()  #make a copy of the initial data
m = np.ones(K0.shape[0])[:N] #initial masses

omegaIJ = build_omegaij(K,m)
omegaI = build_omegai(0.5*K.sum(axis=1),m) #0.5 since K is symmetric

# %%time
'''RG flow'''
for counter in range(3):
    #Find max btw node and edges
    IJmax = omegaIJ.max()
    Imax = omegaI.max()
    maxtype = np.argmax([Imax,IJmax])
    print(Imax,IJmax)
    if False: #if edge
        emax = omegaIJ.data.argmax()
        nzr = K.row # list of nonzero rows and cols pairs
        nzc = K.col # list of nonzero rows and cols pairs
        i0 = nzr[emax];j0 = nzc[emax] #find nodes 
        idx_i0isrow = np.argwhere(nzr==i0) # (i0,j)
        idx_i0iscol = np.argwhere(nzc==i0) # (j,i0)
        idx_j0isrow = np.argwhere(nzr==j0) # (j0,i)
        idx_j0iscol = np.argwhere(nzc==j0) # (i,j0)
        nnidxs_of_i0andj0 = np.unique(np.concatenate((idx_i0isrow,idx_i0iscol,idx_j0isrow,idx_j0iscol))) #links to i0 or j0
        nodes_common_to_i0andj0 = np.intersect1d(nzc[idx_i0isrow],nzc[idx_j0isrow])
        nodes_exclusive_to_i0 = np.setdiff1d(nzc[idx_i0isrow],nzc[idx_j0isrow])
        nodes_exclusive_to_j0 = np.setdiff1d(nzc[idx_j0isrow],nzc[idx_i0isrow])
        m = np.append(m,m[i0]+m[j0]) #add a center of mass node
        g = K.row.max()+1 #define node id of the center of mass
        for j in nodes_exclusive_to_i0:
            idxs = np.intersect1d(np.argwhere(nzc==j),np.argwhere(nzr==i0)) #j idx for (i0,j)
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = expand(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,idxs,m,g)
        for j in nodes_exclusive_to_j0:
            idxs = np.intersect1d(np.argwhere(nzc==j),np.argwhere(nzr==j0)) #(j0,j)
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = expand(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,idxs,m,g)
        for j in nodes_common_to_i0andj0:
            idxs_i0 = np.intersect1d(np.argwhere(nzc==j),np.argwhere(nzr==i0)) #(i0,j)
            idxs_j0 = np.intersect1d(np.argwhere(nzc==j),np.argwhere(nzr==j0)) #(j0,j)
            newdata = K.data[idxs_i0]+K.data[idxs_j0]
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = expand1(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,newdata,m,g,j)
        #remove i0 and j0 from K, omegaIJ
        K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,nnidxs_of_i0andj0)
        #update omegaI
        nn_nodes_ofi0andj0 = np.unique(np.concatenate(([nzc[idx] for idx in idx_i0isrow],[nzr[idx] for idx in idx_i0iscol],
                       [nzc[idx] for idx in idx_j0isrow],[nzr[idx] for idx in idx_j0iscol])))
        omegaI_g = np.array(sum([K.data[idx] for idx in np.argwhere(K.row==g)])*1.0/m[g]).reshape(1,1)
        omegaI = np.append(omegaI,omegaI_g,0)
        for j in np.setdiff1d(nn_nodes_ofi0andj0,[g]):
            omegaI[j] = sum([K.data[idx] for idx in np.argwhere(K.row==j)])*1.0/m[j]
    if True: #if node
        i0 = omegaI.argmax(axis=None)
        nzr = K.row # list of nonzero rows and cols pairs
        nzc = K.col # list of nonzero rows and cols pairs
        #find NN of i0
        idx_i0isrow = np.argwhere(nzr==i0) # idxs of (i0,j)
        idx_i0iscol = np.argwhere(nzc==i0) # idx of (i,i0)
        
        sinks_of_i0 = nzc[idx_i0isrow] # nn j in (i0,j)
        sources_of_i0 = nzr[idx_i0iscol] # nn i in (i,i0)
        
        idxs_i = np.unique(np.concatenate([np.argwhere(nzr==i) for i in sources_of_i0])) #idx of (i,...)
        idxs_j = np.unique(np.concatenate([np.argwhere(nzc==j) for j in sinks_of_i0])) #idx of (...,j)
        
        K_ij_idxs = np.union1d(idxs_i,idxs_j) #idxs of (i,j) present in old K
        
        for idx in K_ij_idxs: 
            i=K.row[idx]
            j=K.col[idx]
            idx_ii0 = np.intersect1d(np.argwhere(nzr==i),np.argwhere(nzc==i0))
            idx_i0j = np.intersect1d(np.argwhere(nzr==i0),np.argwhere(nzc==j))
            rescale = K.data[idx_ii0]*K.data[idx_i0j]*1.0/omegaI[i0]
            if rescale.shape == (1,1):
                K.data[idx] = K.data[idx]+rescale
                omegaIJ.data[idx] = K.data[idx]*(1.0/m[i]+1.0/m[j])
                #symmetrize
                idx_ji = np.intersect1d(np.argwhere(nzr==j),np.argwhere(nzc==i))
                K.data[idx_ji] = K.data[idx]
                omegaIJ.data[idx_ji] = omegaIJ.data[idx]
        #remove i0 from K, omegaIJ
        K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.union1d(idx_i0isrow,idx_i0iscol))
        for j in sinks_of_i0:
            omegaI[j] = sum([K.data[idx] for idx in np.argwhere(K.row==j)])*1.0/m[j]    

0.8048674005584304 2.0
2.098843819917449 3.1565588521733354
2.098843819917449 2.069355452149664


ValueError: need at least one array to concatenate

In [77]:
print(new_ij,existing_ij)

[5448 5449 5480 5580 5614 5615 5616 5617 5633 5634 5635 5636 5663 5796
 5797 5798 5800 5804 5805 5821 5873 5874 5875 5880 5883 5892 5898 5925
 5955 5956 5957 5963 5964 6018 6019 6025 6029 6030 6036 6037 6044 6046
 6047 6142 6143 6144 6145 6146 6147 6148 6149 6150 6151 6261 6265 6279
 6282 6283 6316 6394 6398 6399 6400 6622 6625 6626 6627 6628 6629 6630
 6647 6648 6650 6660 6662 6723 6725 6803 6827 7002 7129 7219] [5799 5801 5802 5803 5806 5876 5877 5878 5879 5881 5882 5893 5894 5895
 5896 5897 5899 5900 5901 5958 5959 5960 5961 5962 5965 5966 5967 6020
 6021 6022 6023 6024 6026 6027 6028 6031 6032 6033 6034 6035 6038 6039
 6040 6255 6256 6257 6258 6259 6260 6262 6263 6264 6275 6276 6277 6278
 6280 6281 6390 6391 6392 6393 6395 6396 6397 6621 6623 6624]


In [ ]:
A,B = expand1(K,omegaIJ,newdata,m,g,j)

In [ ]:
sns.set(style='white', rc={'figure.figsize':(50,50)})

G = nx.from_scipy_sparse_matrix(mat_XY) # if sparse matrix
eset = [(u, v) for (u, v, d) in G.edges(data=True)]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
pos = XY

In [ ]:
nx.draw_networkx_nodes(G, pos,alpha=0.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')
plt.axis('off')
# plt.savefig(str(img_id)+'graph_only.png',bbox_inches='tight')
# plt.close()
